In [1]:
%reload_ext autoreload
%autoreload 3

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import matplotlib.animation as animation
from scipy.stats import binom, t
import matplotlib as mpl
from sklearn import datasets
import itertools

from utils import *

In [2]:
X_0 = np.random.rand(50,2) + 1
X_1 = np.random.rand(50,2) - 1
X = np.concatenate([X_0, X_1])

x_min, x_max = min(X[:,0]), max(X[:,0])
y_min, y_max = min(X[:,1]), max(X[:,1])

In [3]:
W, D_sqinv = compute_G0(X)
L = compute_laplacian(W, D_sqinv)
A = compute_AM_normalised(L, D_sqinv, 3) #3 peut être faible
W, g = compute_Ge(A)

In [4]:
g.compute_A()
g.kruskal_algo()
mat = g.compute_mat()

In [5]:
def build_rand_mat(K, x_min, y_min, x_max, y_max):
    X_H0 = np.random.uniform(size=(K,2)) * np.array([x_max - x_min, y_max - y_min]) + np.array([x_min, y_min])
    W, D_sqinv = compute_G0(X_H0)
    L = compute_laplacian(W, D_sqinv)
    A = compute_AM_normalised(L, D_sqinv, 3) #3 peut être faible
    W, g = compute_Ge(A)
    g.compute_A()
    g.kruskal_algo()
    mat = g.compute_mat()
    return mat, g

In [30]:
def update_deltas(deltas, eps, mat, K, x_min, y_min, x_max, y_max, update_, Q=10):
    res_q = np.zeros((K, Q), dtype=int)
    for q in range(Q):
        res = np.zeros(K, dtype=int)
        mat, g = build_rand_mat(K-1, x_min, y_min, x_max, y_max)
        omega = g.omega
        V = g.V
        for i in range(len(mat)):
            k = int(mat[i, 5]) #hash function pour l'instant c'est juste le nb de sommets 
            pfa = compute_pfa(mat[i, 4] / omega, V , k)
            if pfa < deltas[k]:
                res[k] += 1
        res_q[:, q] = res
    s = np.maximum(np.std(res_q, axis=1), 1e-5)
    m = np.mean(res_q, axis=1)
    upd = t.cdf((m - eps / K) / s * np.sqrt(Q - 1), Q-1) > conf_level
    deltas -= (2 * upd - 1) * update_
    deltas = np.minimum(1, deltas)
    return deltas, res_q


In [29]:
# MC complet
K = 10
deltas = (1 - 1e-12) * np.ones(K)
eps = 0.5
conf_level = 0.05
update_ = 1e-9
Q = 10

deltas,_ = update_deltas(deltas, eps, mat, K, x_min, y_min, x_max, y_max, update_, Q=10)
for i in deltas:
    print(i)

1.0
1.0
0.999999998999
0.999999998999
0.999999998999
0.999999998999
0.999999998999
0.999999998999
0.999999998999
1.0


In [ ]:
K = 10
deltas = (1 - 1e-3) * np.ones(K)
eps = 0.5
conf_level = 0.05
update_ = 1e-4
Q = 10

for i in range(1001):
    deltas, res_q = update_deltas(deltas, eps, mat, K, x_min, y_min, x_max, y_max, update_, Q=10)
    if i % 50 == 0:
        print(i, np.mean(res_q))
print(deltas)

0 0.21
50 0.17
100 0.18
150 0.18
200 0.11
250 0.12
300 0.12
350 0.1
400 0.12
450 0.08
500 0.09
550 0.04
600 0.03
650 0.07
700 0.04
750 0.07
800 0.09
850 0.05
